In [1]:
import os
import sys
sys.path.append("..")
import keras
import keras.backend as K
from WVutils.data import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from datautils.other import *
from models.model import *

/home/deep/pythonenv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Building prefix dict from the default dictionary ...
2018-07-31 09:38:59 AM:DEBUG:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2018-07-31 09:38:59 AM:DEBUG:Loading model from cache /tmp/jieba.cache
Loading model cost 0.587 seconds.
2018-07-31 09:38:59 AM:DEBUG:Loading model cost 0.587 seconds.
Prefix dict has been built succesfully.
2018-07-31 09:38:59 AM:DEBUG:Prefix dict has been built succesfully.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
train = pd.read_csv(Config.cache_path + 'trainset.csv', sep='\t')
val = pd.read_csv(Config.cache_path + 'valset.csv', sep='\t')

In [5]:
from keras.utils import to_categorical
val_label = to_categorical(val.label)

In [13]:
train.fillna("", inplace=True)
val.fillna("",inplace=True)

# TextCNN

In [6]:
from datautils.pre import *
from WVutils.config import *

Process ForkPoolWorker-2:
KeyboardInterrupt
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/deep/pythonenv/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/deep/pythonenv/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/deep/pythonenv/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/deep/pythonenv/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/deep/pythonenv/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/deep/pythonenv/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._ta

In [7]:
batch_size=64
model_name='textcnn'
train_batch_generator = word_cnn_train_batch_generator
print('Loading data')

Loading data


In [8]:
val_word_seq = pickle.load(open(Config.cache_path + 'val_cnn_seq.pkl', 'rb'))

In [9]:
word_embed_weight = np.load(Config.word_embed_weight_path)

In [10]:
model=get_textcnn(Config.word_seq_maxlen, word_embed_weight)

In [12]:
val_word_seq.dtype

dtype('int32')

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1000, 256)    60197376    input_8[0][0]                    
__________________________________________________________________________________________________
time_distributed_8 (TimeDistrib (None, 1000, 256)    65792       embedding_8[0][0]                
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 1000, 256)    1024        time_distributed_8[0][0]         
__________________________________________________________________________________________________
activation

In [45]:
for i in range(8):
    if i == 3:
        K.set_value(model.optimizer.lr, 0.0002)
    if i == 6:
        for l in trainable_layer:
            model.get_layer(l).trainable = True
    model.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 186s 120ms/step - loss: 0.6724 - acc: 0.6077 - val_loss: 0.6296 - val_acc: 0.6320
0.7739130434782608 0.0827906976744186 0.14957983193277308
Epoch 1/1
1546/1546 [==============================] - 187s 121ms/step - loss: 0.5640 - acc: 0.6880 - val_loss: 0.6103 - val_acc: 0.6475
0.5284925232385828 0.9123255813953488 0.6692826068412523
Epoch 1/1
1546/1546 [==============================] - 190s 123ms/step - loss: 0.4974 - acc: 0.7393 - val_loss: 0.4744 - val_acc: 0.7565
0.6551790158912503 0.7958139534883721 0.7186810878924709
Epoch 1/1
1546/1546 [==============================] - 189s 122ms/step - loss: 0.4356 - acc: 0.7834 - val_loss: 0.4295 - val_acc: 0.7910
0.7162308191052518 0.7706976744186047 0.7424666741346477
Epoch 1/1
1546/1546 [==============================] - 187s 121ms/step - loss: 0.4038 - acc: 0.8023 - val_loss: 0.4211 - val_acc: 0.7989
0.7579051383399209 0.7134883720930233 0.7350263536176328
Epoch 1/1
1546/1546 [========

ValueError: No such layer: word_embedding

In [60]:
model.save(Config.cache_path+'txtcnn1.h5')

In [14]:
model = load_model(Config.cache_path+'txtcnn1.h5')

In [16]:
K.set_value(model.optimizer.lr, 0.0001)

In [20]:
for i in range(3):
    model.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 189s 122ms/step - loss: 0.2883 - acc: 0.8695 - val_loss: 0.4127 - val_acc: 0.8065
0.7214518760195758 0.8227906976744186 0.7687961755758366
Epoch 1/1
1546/1546 [==============================] - 191s 124ms/step - loss: 0.2562 - acc: 0.8875 - val_loss: 0.4350 - val_acc: 0.8002
0.7036821705426357 0.8444186046511628 0.7676532769556025
Epoch 1/1
1546/1546 [==============================] - 189s 122ms/step - loss: 0.2297 - acc: 0.9006 - val_loss: 0.5594 - val_acc: 0.7690
0.6450123660346249 0.9097674418604651 0.754848046309696


In [18]:
i = 0
for content in train.content.values:
    if type(content) != str:
        print(content, i)
    i += 1

nan 601
nan 3373
nan 21249
nan 25593
nan 31465
nan 32237
nan 34887
nan 45295
nan 50265
nan 57035
nan 57351
nan 60613
nan 74078
nan 95445
nan 97726


# No TimeDistributed layer

In [24]:
model_noTD=get_textcnn_no_TD(Config.word_seq_maxlen, word_embed_weight)

In [25]:
for i in range(8):
    if i == 3:
        K.set_value(model_noTD.optimizer.lr, 0.0002)
    if i == 6:
        for l in trainable_layer:
            model_noTD.get_layer(l).trainable = True
    model_noTD.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model_noTD.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 183s 119ms/step - loss: 0.6780 - acc: 0.6060 - val_loss: 0.6437 - val_acc: 0.6319
0.7687366167023555 0.08348837209302326 0.15061883784350746
Epoch 1/1
1546/1546 [==============================] - 181s 117ms/step - loss: 0.6143 - acc: 0.6537 - val_loss: 0.5501 - val_acc: 0.6862
0.7892223738062756 0.26906976744186045 0.4013180714533472
Epoch 1/1
1546/1546 [==============================] - 183s 118ms/step - loss: 0.5475 - acc: 0.7036 - val_loss: 0.5002 - val_acc: 0.7364
0.6674641148325359 0.6488372093023256 0.6580188679245284
Epoch 1/1
1546/1546 [==============================] - 181s 117ms/step - loss: 0.4800 - acc: 0.7560 - val_loss: 0.4926 - val_acc: 0.7437
0.6343188826410303 0.8132558139534883 0.7127280138591664
Epoch 1/1
1546/1546 [==============================] - 183s 119ms/step - loss: 0.4484 - acc: 0.7768 - val_loss: 0.4870 - val_acc: 0.7418
0.6297085998578535 0.8241860465116279 0.7139403706688153
Epoch 1/1
1546/1546 [======

NameError: name 'trainable_layer' is not defined

In [26]:
for l in model_noTD.layers:
    l.trainable = True

In [27]:
for i in range(3):
    model_noTD.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model_noTD.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 174s 113ms/step - loss: 0.4031 - acc: 0.8079 - val_loss: 0.4534 - val_acc: 0.7737
0.6968050423820908 0.7455813953488372 0.7203684979215819
Epoch 1/1
1546/1546 [==============================] - 184s 119ms/step - loss: 0.3771 - acc: 0.8243 - val_loss: 0.4546 - val_acc: 0.7693
0.6789195775792038 0.7774418604651163 0.7248482220294883
Epoch 1/1
1546/1546 [==============================] - 182s 118ms/step - loss: 0.3592 - acc: 0.8331 - val_loss: 0.4750 - val_acc: 0.7555
0.6444205238607822 0.8353488372093023 0.7275673485922624


# HAN

In [6]:
val_word_seq = pickle.load(open(Config.cache_path+'val_han_seq.pkl', "rb"))

In [33]:
?get_han

In [7]:
from datautils.pre import *
from WVutils.config import *

In [8]:
train_batch_generator = word_han_train_batch_generator

In [9]:
batch_size=64

In [10]:
val_seq = val_word_seq
word_embed_weight = np.load(Config.word_embed_weight_path)

In [14]:
model_han=get_han(Config.sentence_num, Config.sentence_word_length, word_embed_weight)
model_han.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 45, 48)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 45, 256)           60493360  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 45, 256)           295680    
_________________________________________________________________
attention_4 (Attention)      (None, 256)               301       
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
__________

In [15]:
for i in range(10):
    if i == 3:
        K.set_value(model_han.optimizer.lr, 0.0002)
    if i == 7:
        for l in model_han.layers:
            l.trainable = True
    model_han.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model_han.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 305s 197ms/step - loss: 0.6407 - acc: 0.6338 - val_loss: 0.5842 - val_acc: 0.6782
0.6360057265569077 0.41325581395348837 0.5009867493656611
Epoch 1/1
1546/1546 [==============================] - 313s 203ms/step - loss: 0.5448 - acc: 0.7062 - val_loss: 0.5670 - val_acc: 0.7059
0.6650139448404091 0.4990697674418605 0.5702138966387672
Epoch 1/1
1546/1546 [==============================] - 318s 206ms/step - loss: 0.4807 - acc: 0.7533 - val_loss: 0.5204 - val_acc: 0.7372
0.6614714645885859 0.6711627906976744 0.6662818884912848
Epoch 1/1
1546/1546 [==============================] - 312s 202ms/step - loss: 0.3717 - acc: 0.8240 - val_loss: 0.5002 - val_acc: 0.7635
0.6919077757685352 0.7118604651162791 0.7017423200366805
Epoch 1/1
1546/1546 [==============================] - 311s 201ms/step - loss: 0.3157 - acc: 0.8539 - val_loss: 0.5552 - val_acc: 0.7488
0.6536078352988207 0.7604651162790698 0.7029990325701387
Epoch 1/1
1546/1546 [========

# HCNN

In [19]:
train_batch_generator = word_han_train_batch_generator
model_hcnn = get_hcnn(Config.sentence_num, Config.sentence_word_length, word_embed_weight)
model_hcnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 45, 48)       0                                            
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, 45, 256)      60493360    input_11[0][0]                   
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 43, 256)      196864      time_distributed_5[0][0]         
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 42, 256)      262400      time_distributed_5[0][0]         
__________________________________________________________________________________________________
conv1d_6 (

In [30]:
for i in range(10):
    if i == 5:
        K.set_value(model_hcnn.optimizer.lr, 0.0001)
    if i == 8:
        for l in model_hcnn.layers:
            l.trainable = True
    model_hcnn.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model_hcnn.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 289s 187ms/step - loss: 0.6691 - acc: 0.6080 - val_loss: 0.6429 - val_acc: 0.6425
0.7050279329608938 0.14674418604651163 0.24292589027911451
Epoch 1/1
1546/1546 [==============================] - 293s 190ms/step - loss: 0.5747 - acc: 0.6812 - val_loss: 0.5589 - val_acc: 0.6963
0.6070073644275831 0.6325581395348837 0.6195194169229018
Epoch 1/1
1546/1546 [==============================] - 295s 191ms/step - loss: 0.5211 - acc: 0.7217 - val_loss: 0.5273 - val_acc: 0.7190
0.661847389558233 0.5748837209302325 0.6153080273802116
Epoch 1/1
1546/1546 [==============================] - 294s 190ms/step - loss: 0.4725 - acc: 0.7582 - val_loss: 0.5174 - val_acc: 0.7307
0.6832876712328767 0.58 0.6274213836477988
Epoch 1/1
1546/1546 [==============================] - 294s 190ms/step - loss: 0.4168 - acc: 0.7962 - val_loss: 0.6147 - val_acc: 0.7117
0.8130892956184138 0.34093023255813953 0.4804194658364739
Epoch 1/1
1546/1546 [=====================

# Switch to SPcut embedding

In [31]:
sp_word_embed_weight = np.load(Config.word_embed_weight_path_sp)

In [34]:
val_word_seq = pickle.load(open(Config.cache_path + 'val_cnn_seq.pkl', 'rb'))

In [35]:
train_batch_generator = word_cnn_train_batch_generator

In [36]:
model=get_textcnn(Config.word_seq_maxlen, sp_word_embed_weight)
for i in range(10):
    if i == 4:
        K.set_value(model.optimizer.lr, 0.0001)
    if i == 7:
        for l in model.layers:
            l.trainable = True
    model.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 182s 118ms/step - loss: 0.6823 - acc: 0.6051 - val_loss: 0.6635 - val_acc: 0.6236
0.5531914893617021 0.19348837209302325 0.2866988283942109
Epoch 1/1
1546/1546 [==============================] - 185s 120ms/step - loss: 0.6292 - acc: 0.6397 - val_loss: 0.5911 - val_acc: 0.6514
0.7859778597785978 0.1486046511627907 0.2499511050264033
Epoch 1/1
1546/1546 [==============================] - 186s 120ms/step - loss: 0.5728 - acc: 0.6752 - val_loss: 0.5642 - val_acc: 0.6688
0.8249258160237388 0.19395348837209303 0.3140651478064395
Epoch 1/1
1546/1546 [==============================] - 185s 120ms/step - loss: 0.5447 - acc: 0.6959 - val_loss: 0.5298 - val_acc: 0.7149
0.6313176895306859 0.6506976744186046 0.640861200183234
Epoch 1/1
1546/1546 [==============================] - 186s 120ms/step - loss: 0.5082 - acc: 0.7259 - val_loss: 0.5142 - val_acc: 0.7265
0.6482885366413967 0.6562790697674419 0.6522593320235756
Epoch 1/1
1546/1546 [========

In [37]:
val_word_seq = pickle.load(open(Config.cache_path+'val_han_seq.pkl', "rb"))
train_batch_generator = word_han_train_batch_generator
model_han=get_han(Config.sentence_num, Config.sentence_word_length, word_embed_weight)
for i in range(10):
    if i == 5:
        K.set_value(model_han.optimizer.lr, 0.0001)
    if i == 8:
        for l in model_han.layers:
            l.trainable = True
    model_han.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model_han.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 315s 204ms/step - loss: 0.6271 - acc: 0.6433 - val_loss: 0.6980 - val_acc: 0.6212
0.5097722263041881 0.8067441860465117 0.624763619990995
Epoch 1/1
1546/1546 [==============================] - 313s 202ms/step - loss: 0.5361 - acc: 0.7132 - val_loss: 0.5383 - val_acc: 0.7244
0.6739846322722283 0.5711627906976744 0.6183282980866063
Epoch 1/1
1546/1546 [==============================] - 313s 203ms/step - loss: 0.4754 - acc: 0.7588 - val_loss: 0.4998 - val_acc: 0.7395
0.652692062140881 0.7132558139534884 0.6816312923658184
Epoch 1/1
1546/1546 [==============================] - 313s 202ms/step - loss: 0.4144 - acc: 0.7983 - val_loss: 0.4800 - val_acc: 0.7604
0.674496644295302 0.747906976744186 0.7093074547860609
Epoch 1/1
1546/1546 [==============================] - 315s 204ms/step - loss: 0.3506 - acc: 0.8364 - val_loss: 0.5163 - val_acc: 0.7688
0.8134141990724224 0.5302325581395348 0.6419822610164719
Epoch 1/1
1546/1546 [=============

In [39]:
model_hcnn = get_hcnn(Config.sentence_num, Config.sentence_word_length, word_embed_weight)
for i in range(10):
    if i == 5:
        K.set_value(model_hcnn.optimizer.lr, 0.0001)
    if i == 8:
        for l in model_hcnn.layers:
            l.trainable = True
    model_hcnn.fit_generator(train_batch_generator(train.content.values, train.label.values, batch_size=batch_size),
                       epochs=1, steps_per_epoch=train.shape[0]//batch_size,
                       validation_data = (val_word_seq, val_label))
    pred = np.squeeze(model_hcnn.predict(val_word_seq))
    pre, rec, f1 = score(pred, val_label)
    print(pre, rec, f1)

Epoch 1/1
1546/1546 [==============================] - 290s 188ms/step - loss: 0.6709 - acc: 0.6071 - val_loss: 0.6386 - val_acc: 0.6341
0.5439156818907698 0.396046511627907 0.4583501547570986
Epoch 1/1
1546/1546 [==============================] - 295s 191ms/step - loss: 0.5823 - acc: 0.6779 - val_loss: 0.5977 - val_acc: 0.6588
0.5446676465784746 0.7755813953488372 0.6399309219994243
Epoch 1/1
1546/1546 [==============================] - 295s 191ms/step - loss: 0.5297 - acc: 0.7189 - val_loss: 0.5373 - val_acc: 0.7120
0.6155102040816327 0.7013953488372093 0.6556521739130435
Epoch 1/1
1546/1546 [==============================] - 296s 192ms/step - loss: 0.4678 - acc: 0.7616 - val_loss: 0.5001 - val_acc: 0.7393
0.6479950392724266 0.7290697674418605 0.6861457649376231
Epoch 1/1
1546/1546 [==============================] - 295s 191ms/step - loss: 0.3944 - acc: 0.8113 - val_loss: 0.4725 - val_acc: 0.7629
0.6628175519630485 0.8009302325581396 0.725358045492839
Epoch 1/1
1546/1546 [===========